In [1]:
%cd /mnt/hdd1/jkliu/Projects/ATGEmpirical/

/mnt/hdd1/jkliu/Projects/ATGEmpirical


/mnt/hdd1/jkliu/Projects/ATGEmpirical/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from result_plot.common import *
from core.apk_util import parse_manifest_file


def get_apk_tool_result(apk_path: str, real_transitions: dict):
    apk_tool_transitions = {}
    package_name, _, declared_activities, exported_activities = parse_manifest_file(apk_path)
    apk_name, tool_transitions = get_tool_result(apk_path, declared_activities)
    if tool_transitions is None:
        return
    if apk_name not in real_transitions:
        return
    for tool in tools:
        if tool in tool_transitions:
            tool_transition = tool_transitions[tool]
            apk_tool_transitions[tool] = tool_transition
    return apk_tool_transitions


Updated PATH: /usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/mnt/hdd1/jkliu/Applications/ncurses  # 你自己的 ncurses 目录/bin:/mnt/hdd1/jkliu/Applications/zsh/bin:/mnt/hdd1/jkliu/Applications/Android/cmdline-tools/latest/bin:/mnt/hdd1/jkliu/Applications/Android/emulator:/mnt/hdd1/jkliu/Applications/Android/platform-tools:/mnt/hdd1/jkliu/Applications/Android/tools:/mnt/hdd1/jkliu/Applications/Android/tools/bin:/mnt/hdd1/jkliu/Applications/Android/build-tools/35.0.0
/mnt/hdd1/jkliu/Applications/Android/build-tools/35.0.0/aapt
Successfully added aapt to PATH


In [3]:
def get_all_results(real_transitions: dict):
    all_transitions = {}
    for tool in tools:
        all_transitions[tool] = set()
    all_transitions['ground truth'] = set()
    all_apks = real_transitions.keys()
    for apk in all_apks:
        real_transition = real_transitions[apk]
        all_transitions['ground truth'].update(real_transition)
        result = get_apk_tool_result(apk, real_transitions)
        if result is not None:
            for tool in tools:
                if tool in result:
                    transitions = result[tool]
                    all_transitions[tool].update(transitions)
    return all_transitions


In [4]:
real_transitions = get_real_transitions()
set_current_version("first_round_no_internet")
all_transitions = get_all_results(real_transitions)

extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/com.gpl.rpg.AndorsTrail_75.apk
extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/com.darkempire78.opencalculator_42.apk
extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/rocks.poopjournal.vacationdays_9.apk
extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/org.documentfoundation.libreoffice_38.apk
extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/org.xphnx.ameixamonochrome_4100.apk
extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/com.github.axet.audiorecorder_377.apk
extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/de.k3b.android.toGoZip_30.apk
extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/com.simplemobiletools.calculator_61.apk
extracting summaries for /mnt/hdd1/jkliu/Projects/ATGEmpirical/apks/fdroid/com.jarsilio.android.waveup_121.

In [5]:
all_transitions


{'ape': {'at.jclehner.rxdroid/.BackupActivity -> at.jclehner.rxdroid/.SettingsActivity',
  'at.jclehner.rxdroid/.DoseTimePreferenceActivity2 -> at.jclehner.rxdroid/.DrugListActivity2',
  'at.jclehner.rxdroid/.DoseTimePreferenceActivity2 -> at.jclehner.rxdroid/.SettingsActivity',
  'at.jclehner.rxdroid/.DrugEditActivity2 -> at.jclehner.rxdroid/.DrugListActivity2',
  'at.jclehner.rxdroid/.DrugListActivity2 -> at.jclehner.rxdroid/.DrugEditActivity2',
  'at.jclehner.rxdroid/.DrugListActivity2 -> at.jclehner.rxdroid/.SettingsActivity',
  'at.jclehner.rxdroid/.SettingsActivity -> at.jclehner.rxdroid/.BackupActivity',
  'at.jclehner.rxdroid/.SettingsActivity -> at.jclehner.rxdroid/.DoseTimePreferenceActivity2',
  'at.jclehner.rxdroid/.SettingsActivity -> at.jclehner.rxdroid/.DrugListActivity2',
  'at.tomtasche.reader/.ui.activity.MainActivity -> at.tomtasche.reader/com.nononsenseapps.filepicker.FilePickerActivity',
  'at.tomtasche.reader/com.nononsenseapps.filepicker.FilePickerActivity -> at.

In [ ]:
def calculate_precision_score(gt_transitions: list, tool_transitions: list):
    if len(gt_transitions) == 0:
        return 0
    if len(tool_transitions) == 0:
        return 0
    correct = 0
    for transition in tool_transitions:
        if transition in gt_transitions:
            correct += 1
    return correct / len(tool_transitions)

def calculate_recall_score(gt_transitions: list, tool_transitions: list):
    if len(gt_transitions) == 0:
        return 0
    if len(tool_transitions) == 0:
        return 0
    correct = 0
    for transition in tool_transitions:
        if transition in gt_transitions:
            correct += 1
    return correct / len(gt_transitions)

def calculate_f1_score(precision: float, recall: float):
    if precision == 0 or recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)